In [ ]:
# 'wiki' index 생성 - 1536 dim

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

In [2]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

In [3]:
index_name = "wiki"
for idx in pc.list_indexes():
    # 인덱스 이름이 "wiki"와 일치하는 경우 해당 인덱스를 삭제합니다.
    if idx.name == index_name:
        pc.delete_index(idx.name)

In [4]:
pc.create_index(
    name=index_name,
    dimension=1536,  # 모델 차원
    metric="cosine",  # 모델 메트릭
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "wiki",
    "metric": "cosine",
    "host": "wiki-506yx3r.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [5]:
# 'wiki' 인덱스를 가져옵니다.
index = pc.Index(index_name)

index.describe_index_stats()

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

# data load

In [6]:
%pip install -qU datasets

Note: you may need to restart the kernel to use updated packages.


In [9]:
from datasets import load_dataset
data = load_dataset("wikimedia/wikipedia", "20231101.ko", split='train[:100]')

In [10]:
data

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 100
})

In [11]:
data[0]

{'id': '5',
 'url': 'https://ko.wikipedia.org/wiki/%EC%A7%80%EB%AF%B8%20%EC%B9%B4%ED%84%B0',
 'title': '지미 카터',
 'text': '제임스 얼 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령(1977년~1981년)이다.\n\n생애\n\n어린 시절 \n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n\n정계 입문 \n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n\n대통령 재임 \n\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받았는데 제럴드 포드 대통령을 누르고 당선되었다.\n\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n\n외교 정책 \n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으

In [12]:
len(data)

100

In [13]:
for rec in data:
  if len(rec['text'])>35000:
    print('**')

**
**


In [14]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model='text-embedding-3-small')
embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000001CD5CEBBA40>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001CD5EA37590>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
  chunk_size=400,
  chunk_overlap =20,
  length_function = len, \
  separators=["\n\n", "\n", " ", ""]  #자르는 순서 문자
)
splitter

In [19]:
import json
def calculate_metadata_size(metadata):
  return len(json.dumps(metadata, ensure_ascii=False).encode('utf-8'))

In [22]:
from uuid import uuid4
import time
texts = []
metas = []
count = 0
batch_size = 20

In [23]:
for i, sample in enumerate(data):
  full_text = sample['text'] #전문
  
  #메타데이터 구성
  metadata = {
    "wiki_id": str(sample['id']),
    "url": sample['url'],
    "title": sample['title'] #[:100]
  }
  
  # 스플릿터로 자르기
  chunks = splitter.split_text(full_text)
  #print(len(chunks))
  
  for i, chunk in enumerate(chunks):
    record = {
      'chunk_id':i,
      'full_text':full_text,
      **metadata
    }
    metadata_size = calculate_metadata_size(record)
    if metadata_size > 35000:
      continue
    
    texts.append(chunk)
    metas.append(record)
    count += 1
    
    # 청크 배치의 임베딩 -> 적재
    if count % batch_size == 0:
      ids = [str(uuid4()) for _ in range(len(texts))]
      embeddings = embedding.embed_documents(texts) #20개 청크의 임베딩 수행
      index.upsert(
        vectors=zip(ids, embeddings, metas),
        namespace='wiki-ns1'
      )
      texts=[]
      metas=[]
      time.sleep(1)

In [25]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'wiki-ns1': {'vector_count': 1160}},
 'total_vector_count': 1160,
 'vector_type': 'dense'}

# 질의 임베딩

In [26]:
question = ["벨기에는 어디에 있나요?"]
emb_question = embedding.embed_documents(question)
emb_question

[[0.0029637617990374565,
  0.0386190339922905,
  0.08355793356895447,
  0.011749008670449257,
  -0.01028568483889103,
  -0.015439130365848541,
  0.011886858381330967,
  0.01581026427447796,
  -0.02797282114624977,
  -0.021971071138978004,
  -0.01926710270345211,
  0.058151230216026306,
  0.042224325239658356,
  -0.03075101599097252,
  0.0014792298898100853,
  0.005582899786531925,
  -0.0692640095949173,
  0.016764605417847633,
  0.02687002532184124,
  0.02194986306130886,
  -0.02337077260017395,
  -0.022904204204678535,
  -0.032256755977869034,
  0.04869264364242554,
  0.042542438954114914,
  0.007162865716964006,
  0.015576980076730251,
  0.0003840563294943422,
  0.006823544390499592,
  -0.03618016093969345,
  0.02264971286058426,
  -0.03255366161465645,
  0.02139846608042717,
  -0.05034683644771576,
  0.014198485761880875,
  0.00985092855989933,
  0.04504493623971939,
  -0.04411180317401886,
  0.017856797203421593,
  -0.023498017340898514,
  -0.006081278435885906,
  0.005150794982910

# Retrieval

In [28]:
# 파인콘 질의 실행
q_result = index.query(
  namespace='wiki-ns1',
  vector=emb_question,
  top_k=5,
  include_vector = False,
  include_metadata=True
)
q_result

{'matches': [{'id': 'eba1897c-7877-4b38-bda3-33d7dac5962a',
              'metadata': {'chunk_id': 11.0,
                           'full_text': '엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 '
                                        '없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. '
                                        '통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태의 수의 '
                                        '로그로 생각할 수 있다. 엔트로피는 일반적으로 보존되지 않고, '
                                        '열역학 제2법칙에 따라 시간에 따라 증가한다. 독일의 물리학자 '
                                        '루돌프 클라우지우스가 1850년대 초에 도입하였다. 대개 기호로 '
                                        '라틴 대문자 S를 쓴다.\n'
                                        '\n'
                                        '정의 \n'
                                        '엔트로피에는 열역학적 정의와 통계학적인 정의, 두 가지의 관련된 '
                                        '정의가 있다. 역사적으로, 고전 열역학적 정의가 먼저 발전하였다. '
                                        '고전 열역학적인 관점에서, 그 이론은 원자나 분자 같은 수많은 '
                                      

In [30]:
result_ids = [ r.id for r in q_result.matches]
result_ids

['eba1897c-7877-4b38-bda3-33d7dac5962a',
 'f7afb2e3-5336-438b-bb9b-956063456343',
 'aff8b363-5e2b-4599-bd25-cf83d59f8752',
 '3c996bc6-b210-43b8-9c49-fc7530e9f7bf',
 '7e95d3ac-8f14-4a4f-b4e2-b384d8b79ef1']

In [33]:
for r in q_result.matches:
  print(r.score, r.metadata)

0.306882858 {'chunk_id': 11.0, 'full_text': '엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. 통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태의 수의 로그로 생각할 수 있다. 엔트로피는 일반적으로 보존되지 않고, 열역학 제2법칙에 따라 시간에 따라 증가한다. 독일의 물리학자 루돌프 클라우지우스가 1850년대 초에 도입하였다. 대개 기호로 라틴 대문자 S를 쓴다.\n\n정의 \n엔트로피에는 열역학적 정의와 통계학적인 정의, 두 가지의 관련된 정의가 있다. 역사적으로, 고전 열역학적 정의가 먼저 발전하였다. 고전 열역학적인 관점에서, 그 이론은 원자나 분자 같은 수많은 성분들로 이루어져 있고, 학설의 안정성은 그러한 성분들의 평균적인 열특성으로 설명된다.\n이론을 구성하는 성분의 세부적인 성분들은 직접적으로 보이지 않는다. 그러나 그 특성은 온도, 압력, 엔트로피, 열용량과 같은 눈으로 볼 수 있는 평균적인 지표들에 의해 설명된다. 그 이론의 특성에 관한 고전적인 정의는 평형 상태임을 가정하였다. 엔트로피의 고전 열역학적 정의는 최근 비평형 열역학의 영역으로까지 확대되었다. 이후 엔트로피를 포함한 열특성은 눈으로 볼 수 없는 미시세계에서의 움직임에 대한 정역학적인 관점에서 새롭게 정의되었다. 그 예로 처음에는 기체로 여겨졌지만 시간이 지난 후에 광자, 음자, 스핀과 같은 양자 역학적으로 생각되는 뉴턴 입자가 있다. 이 이론의 특성에 대한 통계학적 설명은 고전적인 열역학을 사용하여 이론의 특성을 정의하는 것이 몇몇 변수의 영향을 받는 이론의 최종적인 상태를 예측하는데 있어서 점점 신뢰할 수 없는 예측 기술이 되어감으로 인하여 필수적인 것이 되었다.\n\n열역학적 정의 \n고전적 열역학에서는 엔트로피 S의 절대적 값은 정의할 수 없고, 대신 그 상대적 변화만 정의한다. 열적 평형을 이뤄 온도가 인 계에 열 를 가하였다고 하자. 이 경우 엔트로피의 증가

# 랭체인의 api 로 결과 비교하기

In [34]:
question

['벨기에는 어디에 있나요?']

In [35]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embedding, text_key='full_text')
docs = vector_store.similarity_search(query=question[0], k=5, namespace='wiki-ns1')
docs

[Document(id='eba1897c-7877-4b38-bda3-33d7dac5962a', metadata={'chunk_id': 11.0, 'title': '엔트로피', 'url': 'https://ko.wikipedia.org/wiki/%EC%97%94%ED%8A%B8%EB%A1%9C%ED%94%BC', 'wiki_id': '158'}, page_content='엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. 통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태의 수의 로그로 생각할 수 있다. 엔트로피는 일반적으로 보존되지 않고, 열역학 제2법칙에 따라 시간에 따라 증가한다. 독일의 물리학자 루돌프 클라우지우스가 1850년대 초에 도입하였다. 대개 기호로 라틴 대문자 S를 쓴다.\n\n정의 \n엔트로피에는 열역학적 정의와 통계학적인 정의, 두 가지의 관련된 정의가 있다. 역사적으로, 고전 열역학적 정의가 먼저 발전하였다. 고전 열역학적인 관점에서, 그 이론은 원자나 분자 같은 수많은 성분들로 이루어져 있고, 학설의 안정성은 그러한 성분들의 평균적인 열특성으로 설명된다.\n이론을 구성하는 성분의 세부적인 성분들은 직접적으로 보이지 않는다. 그러나 그 특성은 온도, 압력, 엔트로피, 열용량과 같은 눈으로 볼 수 있는 평균적인 지표들에 의해 설명된다. 그 이론의 특성에 관한 고전적인 정의는 평형 상태임을 가정하였다. 엔트로피의 고전 열역학적 정의는 최근 비평형 열역학의 영역으로까지 확대되었다. 이후 엔트로피를 포함한 열특성은 눈으로 볼 수 없는 미시세계에서의 움직임에 대한 정역학적인 관점에서 새롭게 정의되었다. 그 예로 처음에는 기체로 여겨졌지만 시간이 지난 후에 광자, 음자, 스핀과 같은 양자 역학적으로 생각되는 뉴턴 입자가 있다. 이 이론의 특성에 대한 통계학적 설명은 고전적인 열역학을 사용하여 이론의 특성을 정의하는 것이 몇몇 변수의 영향을 받는 이론의 최종적인 상태를 예측하

#질문의 응답 결과가 동일한지 검증합니다.

In [ ]:
question=[
  "",
  "",
  "",
]

# Augmentation